In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [13]:
df_f = pd.read_excel('/Users/tb/Desktop/Google/MSFM UChicago/PM&RM/HW5/factor_pricing_data.xlsx', sheet_name = 'factors (excess returns)').rename(columns={'Unnamed: 0': 'Date'}).set_index('Date')
# df_f = df_f0.drop('RF', axis = 1)
df_f_80 = df_f[:'1980']
df_f_81_01 = df_f['1981':'2001']
df_f_02_22 = df_f['2002':]
df_f_15_22 = df_f['2015':]
df_f.head()

,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0551,0.0183,0.0175,-0.0170,0.0164,0.0755
1980-02-29,-0.0122,-0.0157,0.0061,0.0004,0.0268,0.0788
1980-03-31,-0.1290,-0.0693,-0.0101,0.0146,-0.0119,-0.0955
1980-04-30,0.0397,0.0105,0.0106,-0.0210,0.0029,-0.0043
1980-05-31,0.0526,0.0211,0.0038,0.0034,-0.0031,-0.0112


In [15]:
#2.1
def summary_stats(df, annual_fac):
    report = pd.DataFrame()
    report['Mean'] = df.mean() * annual_fac
    report['Vol'] = df.std() * np.sqrt(annual_fac)
    report['Sharpe'] = report['Mean'] / report['Vol']
    # report['VaR_0.05'] = df.quantile(0.05)
    return round(report, 4)

print('1926 - 1980')    
print(summary_stats(df_f_80, 12))
print('1981 - 2001')    
print(summary_stats(df_f_81_01, 12))
print('2002 - 2022')    
print(summary_stats(df_f_02_22, 12))


1926 - 1980
       Mean     Vol  Sharpe
MKT  0.2029  0.2037  0.9960
SMB  0.0537  0.1088  0.4935
HML -0.1987  0.1178 -1.6874
RMW  0.1078  0.0730  1.4760
CMA -0.0977  0.0769 -1.2699
UMD  0.3078  0.2346  1.3121
1981 - 2001
       Mean     Vol  Sharpe
MKT  0.0773  0.1574  0.4908
SMB  0.0014  0.1097  0.0131
HML  0.0637  0.1113  0.5727
RMW  0.0469  0.0917  0.5113
CMA  0.0531  0.0773  0.6874
UMD  0.1017  0.1451  0.7008
2002 - 2022
       Mean     Vol  Sharpe
MKT  0.0833  0.1540  0.5409
SMB  0.0211  0.0901  0.2337
HML  0.0017  0.1045  0.0161
RMW  0.0397  0.0747  0.5313
CMA  0.0194  0.0642  0.3023
UMD  0.0170  0.1581  0.1078


In [16]:
print('1980 - 2022')    
print(summary_stats(df_f, 12))
print('2015 - 2022')  
print(summary_stats(df_f_15_22, 12))


1980 - 2022
       Mean     Vol  Sharpe
MKT  0.0831  0.1567  0.5305
SMB  0.0122  0.1005  0.1211
HML  0.0275  0.1088  0.2529
RMW  0.0448  0.0834  0.5376
CMA  0.0333  0.0715  0.4652
UMD  0.0655  0.1545  0.4241
2015 - 2022
       Mean     Vol  Sharpe
MKT  0.1069  0.1602  0.6676
SMB -0.0058  0.0977 -0.0590
HML -0.0197  0.1325 -0.1488
RMW  0.0395  0.0712  0.5553
CMA  0.0022  0.0796  0.0282
UMD  0.0255  0.1368  0.1865


2.2
a) Yes, for the whole history (198001 - 202208) all factors have positive risk premium.
b) Since 2015 till now, size factor and value factor have negative risk premium. All the other factors have posivtive risk premium. 

In [9]:
#2.3
corr_mat = round(df_f.corr(),4)
print('1980 - 2022')
corr_mat

1980 - 2022


,MKT,SMB,HML,RMW,CMA,UMD
MKT,1.0000,0.2263,-0.2221,-0.2554,-0.3819,-0.1677
SMB,0.2263,1.0000,-0.0721,-0.4143,-0.0642,-0.0304
HML,-0.2221,-0.0721,1.0000,0.2295,0.6725,-0.2349
RMW,-0.2554,-0.4143,0.2295,1.0000,0.1155,0.0753
CMA,-0.3819,-0.0642,0.6725,0.1155,1.0000,-0.0122
UMD,-0.1677,-0.0304,-0.2349,0.0753,-0.0122,1.0000


a) The contruction method succeed in keeping correlations small for every factors pairs except for CMA and HML pair.
b) Yes, this seems to be the case because HML has high correlation with CMA. Both factors base on the fundamental: firms with high book value tend to invest conservatively.

In [10]:
def tangency (ret_tilda, diag_sigma = False): # Note 1: page 40

    """ Compute tangency portfolio given "excess" (total return)
        Return the matrix of avg return and variance-covariance matrix
        
    Parameters

    ----------

    diag_sigma: bool

        When `True`, set the off diagonal elements of the variance-covariance

        matrix to zero.
    """

    sigma = ret_tilda.cov()

    N = sigma.shape[0]

    sigma_adj = sigma.copy()

    if diag_sigma:

        sigma_adj.loc[:,:] = np.diag(np.diag(sigma_adj))
    
    mu_tilda = ret_tilda.mean()

    sigma_inv = np.linalg.inv(sigma_adj)

    wt = sigma_inv @ mu_tilda / (np.ones(N) @ sigma_inv @ mu_tilda)


    omega_tangency = pd.Series(wt, index = mu_tilda.index)

    return omega_tangency, mu_tilda, sigma_adj

In [12]:
#2.4
omega_tangency, mu_tilda, sigma_adj = tangency(df_f)
omega_tangency


MKT    0.201062
SMB    0.081551
HML   -0.047037
RMW    0.288377
CMA    0.377449
UMD    0.098597
dtype: float64

a) CMA, RMW, MKT are the most important facttors. HML, SMB and UMD are the least important facttors.
b) Yes, CMA has one of the lower mean returns but the highest allocation.

In [18]:
omega_tangency, mu_tilda, sigma_adj = tangency(df_f[['MKT','SMB', 'HML', 'UMD']])
omega_tangency

MKT    0.331433
SMB    0.006051
HML    0.362221
UMD    0.300295
dtype: float64

c) HML, MKT and UMD are the most important facttors. SMB are the least important facttors.
We can conclude that the importance of these styles is very much based on correlation between the factors.


In [19]:
#3.1

df_p = pd.read_excel('/Users/tb/Desktop/Google/MSFM UChicago/PM&RM/HW5/factor_pricing_data.xlsx', sheet_name = 'portfolios (excess returns)').rename(columns={'Unnamed: 0': 'Date'}).set_index('Date')
df_p.head()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0050,0.0283,0.0084,0.1024,-0.0143,0.0999,0.0354,0.0352,0.0048,0.0032,...,0.0159,0.0876,0.0463,-0.0116,0.0458,-0.0279,0.0258,0.0751,0.0299,0.0665
1980-02-29,0.0111,-0.0610,-0.0966,-0.0319,-0.0569,-0.0314,-0.0527,-0.0788,-0.0556,-0.0140,...,-0.0079,-0.0535,-0.0339,-0.0633,-0.0638,-0.0855,-0.0960,-0.0314,-0.0275,-0.0267
1980-03-31,-0.2244,-0.1116,-0.0167,-0.1464,-0.0192,-0.1281,-0.0817,-0.1278,-0.0565,-0.0664,...,-0.0821,-0.1511,-0.1106,-0.0922,-0.1443,-0.0563,-0.0883,-0.2441,-0.1245,-0.1728
1980-04-30,0.0451,0.0766,0.0232,0.0305,0.0831,-0.0521,0.0775,0.0182,0.0304,0.0113,...,0.0419,-0.0097,-0.0300,0.0351,0.0522,0.0729,0.0532,0.0997,0.0448,0.0762
1980-05-31,0.0637,0.0792,0.0457,0.0895,0.0814,0.0512,0.0324,0.0876,0.0560,0.0064,...,0.0565,0.1060,0.1147,0.0868,0.1127,0.0577,0.0557,0.1040,0.0839,0.0684


In [39]:
models = dict({'CAPM' : ['MKT'],
'FFM_3' : ['MKT','SMB','HML'],
'FFM_5' : ['MKT','SMB','HML','RMW','CMA'],
'AQR' : ['MKT','HML','RMW','UMD'] })

p_data = df_p.join(df_f)

aqr_report = pd.DataFrame(index=df_p.columns)
rhs = sm.add_constant(p_data[models['AQR']])

for portf in df_p.columns:
    lhs = p_data[portf]
    res = sm.OLS(lhs, rhs, missing='drop').fit()
    aqr_report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
    aqr_report.loc[portf, 'R^2'] = res.rsquared 
    # aqr_report.loc[portf, 'beta_mkt'] = res.params['MKT']
    # aqr_report.loc[portf, 'beta_hml'] = res.params['HML'] 
    # aqr_report.loc[portf, 'beta_rmw'] = res.params['RMW']     
    # aqr_report.loc[portf, 'beta_umd'] = res.params['UMD']                      
    # aqr_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
    # aqr_report.loc[portf, 'treynor_ratio'] = 12 * aqr_data[portf].mean() / res.params['MKT']
aqr_report

,alpha_hat,R^2
Agric,0.015617,0.330166
Food,0.015205,0.468121
Soda,0.023801,0.309781
Beer,0.026757,0.424796
Smoke,0.039908,0.257474
Toys,-0.027704,0.503276
Fun,0.027091,0.615578
Books,-0.029218,0.688613
Hshld,-0.000890,0.568142
Clths,-0.001364,0.618474


In [22]:
#3.1.b
MAE_alpha = (100 * aqr_report['alpha_hat']).abs().mean()
print('AQR_MAE = {:.2f} %'.format(MAE_alpha))

AQR_MAE = 2.35 %


if the pricing model worked, the alpha estimates should be small because alphas should be 0 if it is the tangency portfolio. Yes, the MAE is small and support the pricing model.

In [43]:
#3.2

for k, model in models.items():
    report = pd.DataFrame(index=df_p.columns)
    rhs = sm.add_constant(p_data[model])
    
    for portf in df_p.columns:
        lhs = p_data[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
    MAE_alpha = (100 * report['alpha_hat']).abs().mean()
    print(k +'_MAE = {:.2f} %'.format(MAE_alpha))

CAPM_MAE = 2.15 %
FFM_3_MAE = 2.54 %
FFM_5_MAE = 3.25 %
AQR_MAE = 2.35 %


CAPM fits the best as it has the lowest MAE.
3.3 The market factor seems very important for pricing as all models include it and the CAPM performs the best. I think Fama and French should consider using the momentum factor as AQR uses it and their model performs better in terms of MAE.

In [60]:
#3.4
R2 = pd.DataFrame()
for k, model in models.items():
    globals()[f"{k}_report"] = pd.DataFrame(index=df_p.columns)
    rhs = sm.add_constant(p_data[model])

    for portf in df_p.columns:
        lhs = p_data[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        globals()[f"{k}_report"].loc[portf, 'R^2'] = res.rsquared 
        R2[f'{k}_R^2'] = globals()[f"{k}_report"][['R^2']].mean().map('{:.2f}'.format)
    # print(k +'_R^2 = {:.2f} '.format(R2_mean))
R2

,CAPM_R^2,FFM_3_R^2,FFM_5_R^2,AQR_R^2
R^2,0.53,0.57,0.60,0.58


These models do not lead to high time-series  stats. Thus, they would not be good in a Linear Factor Decomposition of the assets.

In [80]:
def ts_betas(df, factor_df, factors, intercept=False):
    if intercept == True:
        res = pd.DataFrame(data = None, index = df.columns, columns = ['alpha'])
        res[factors] = None
    else:
        res = pd.DataFrame(data = None, index = df.columns, columns = factors)
    
    for port in df.columns:
        y = df[port]
        if intercept == True:
            X = sm.add_constant(factor_df[factors])
        else:
            X = factor_df[factors]
        model = sm.OLS(y, X).fit()
        res.loc[port] = model.params
    
    return res

def cross_section(df, factor_df, factors, ts_int=True, annualization=12):
    betas = ts_betas(df, factor_df, factors, intercept=ts_int)
    res = pd.DataFrame(data = None, index = betas.index, columns = factors)
    res['Predicted'] = None
    res['Actual'] = None
    
    for port in res.index:
        res.loc[port, factors] = betas.loc[port]
        prem = (betas.loc[port] * factor_df[factors]).sum(axis=1).mean() * annualization
        res.loc[port,['Predicted','Actual']] = prem, df[port].mean() * annualization
    
    return res

def cross_premia(df_cs, factors):
    y = df_cs['Actual'].astype(float)
    X = df_cs[factors].astype(float)

    return sm.OLS(y,X).fit().params.to_frame('CS Premia')

def cross_premia_mae(df_cs, factors, model):
    y = df_cs['Actual'].astype(float)
    X = df_cs[factors].astype(float)

    print(model + ' MAE: ' + str(round(sm.OLS(y,X).fit().resid.abs().mean(), 4)))
    return


In [85]:
#3.5

for k, model in models.items():
    globals()[f"{k}_cs"] = cross_section(p_data.iloc[:, :49], p_data.iloc[:, 49:55], model, ts_int=True)


AQR_cs.head()

,MKT,HML,RMW,UMD,Predicted,Actual
Agric,0.820943,0.155745,-0.022301,0.087183,0.077238,0.092855
Food,0.682631,0.163392,0.525506,0.034443,0.087063,0.102267
Soda,0.791106,0.20735,0.488692,-0.097353,0.087004,0.110805
Beer,0.727232,0.012689,0.604952,0.076292,0.092927,0.119684
Smoke,0.722708,0.211976,0.656358,-0.040284,0.092704,0.132612


In [86]:
(p_data.iloc[:, 49:55].mean()*12).to_frame('TS Premia')

,TS Premia
MKT,0.083123
SMB,0.012169
HML,0.027523
RMW,0.044845
CMA,0.033260
UMD,0.065513


In [87]:
cross_premia(FFM_3_cs, models['FFM_3'])

,CS Premia
MKT,0.101003
SMB,-0.065920
HML,-0.017297


In [88]:
cross_premia(FFM_5_cs, models['FFM_5'])

,CS Premia
MKT,0.094775
SMB,-0.058725
HML,-0.035406
RMW,0.036789
CMA,-0.015450


In [89]:
cross_premia(AQR_cs, models['AQR'])

,CS Premia
MKT,0.086563
HML,-0.040885
RMW,0.045500
UMD,0.055284


The MKT and RMW factors are similar to the sample averages, but the other cross-sectionally estimated premia vary quite a bit.

In [90]:
for k, model in models.items():
    cross_premia_mae(globals()[f"{k}_cs"] , model, k)

CAPM MAE: 0.0214
FFM_3 MAE: 0.0161
FFM_5 MAE: 0.0136
AQR MAE: 0.0172


In [92]:
for k, model in models.items():
    report = pd.DataFrame(index=df_p.columns)
    rhs = sm.add_constant(p_data[model])
    
    for portf in df_p.columns:
        lhs = p_data[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
    MAE_alpha = (100 * report['alpha_hat']).abs().mean()
    print(k +'_MAE = {:.2f} %'.format(MAE_alpha))

CAPM_MAE = 2.15 %
FFM_3_MAE = 2.54 %
FFM_5_MAE = 3.25 %
AQR_MAE = 2.35 %


CAPM MAE is close but other MAEs are quite different between MAE from cross sectional regression and MAE from time-series alpha.